In [1]:
import pandas as pd
import numpy as np
from scipy.stats import lognorm
import matplotlib.pyplot as plt
import shared

ve_or_ju = "ju"

In [2]:
running_order = pd.read_csv(f"data/running_order_2019_with_estimates_{ve_or_ju}.tsv", delimiter="\t")
running_order.head()

,Unnamed: 0,team_id,team,team_base_name,team_country,leg,leg_dist,name,leg_nro,orig_name,num_runs,pred_log_mean,pred_log_std,predicted,log_q_low,log_q_high,log_std,log_std_fixed,final_pace_mean,final_pace_std
0,0,28,OLV Baselland 1,OLV Baselland,SUI,1,10.9,timon schweizer,1,Timon Schweizer,4.0,1.935,0.129,4.281278,1.927112,1.960336,0.016612,0.100000,1.935,0.129
1,1,28,OLV Baselland 1,OLV Baselland,SUI,2,10.5,kaspar hägler,2,Kaspar Hägler,2.0,1.842,0.051,6.382389,1.775996,1.914535,0.069270,0.100000,1.842,0.100
2,2,28,OLV Baselland 1,OLV Baselland,SUI,3,13.2,fabian hertner,3,Fabian Hertner,6.0,1.746,0.070,6.347555,1.781784,1.980862,0.099539,0.100000,1.746,0.070
3,3,28,OLV Baselland 1,OLV Baselland,SUI,4,7.3,jannis schönleber,4,Jannis Schönleber,2.0,1.949,0.118,6.251983,1.908036,2.094361,0.093162,0.100000,1.949,0.100
4,4,28,OLV Baselland 1,OLV Baselland,SUI,5,7.8,stefan lombriser,5,Stefan Lombriser,4.0,2.007,0.110,6.963056,1.826875,2.038693,0.105909,0.105909,2.007,0.110


In [3]:
order = running_order[["team_id", "team", "leg", "leg_dist", "orig_name", "final_pace_mean", "final_pace_std"]]


In [4]:
order = order.rename(index=str, columns={"orig_name": "name", "leg_dist": "dist", "final_pace_mean": "log_mean", "final_pace_std": "log_std"})

order.head()



,team_id,team,leg,dist,name,log_mean,log_std
0,28,OLV Baselland 1,1,10.9,Timon Schweizer,1.935,0.129
1,28,OLV Baselland 1,2,10.5,Kaspar Hägler,1.842,0.100
2,28,OLV Baselland 1,3,13.2,Fabian Hertner,1.746,0.070
3,28,OLV Baselland 1,4,7.3,Jannis Schönleber,1.949,0.100
4,28,OLV Baselland 1,5,7.8,Stefan Lombriser,2.007,0.110


In [5]:
by_teams = order.set_index(["team_id", "leg"]).unstack()
by_teams.head()

team                           \
leg                            1                        2   
team_id                                                     
28               OLV Baselland 1          OLV Baselland 1   
36          IK Hakarpspojkarna 1     IK Hakarpspojkarna 1   
87       Kouvolan Suunnistajat 1  Kouvolan Suunnistajat 1   
96               OLV Baselland 2          OLV Baselland 2   
97          IK Hakarpspojkarna 2     IK Hakarpspojkarna 2   

                                                           \
leg                            3                        4   
team_id                                                     
28               OLV Baselland 1          OLV Baselland 1   
36          IK Hakarpspojkarna 1     IK Hakarpspojkarna 1   
87       Kouvolan Suunnistajat 1  Kouvolan Suunnistajat 1   
96               OLV Baselland 2          OLV Baselland 2   
97          IK Hakarpspojkarna 2     IK Hakarpspojkarna 2   

                                                           \
leg                            5                        6   
team_id                                                     
28               OLV Baselland 1          OLV Baselland 1   
36          IK Hakarpspojkarna 1     IK Hakarpspojkarna 1   
87       Kouvolan Suunnistajat 1  Kouvolan Suunnistajat 1   
96               OLV Baselland 2          OLV Baselland 2   
97          IK Hakarpspojkarna 2     IK Hakarpspojkarna 2   

                                  dist              ... log_mean         \
leg                            7     1     2     3  ...        5      6   
team_id                                             ...                   
28               OLV Baselland 1  10.9  10.5  13.2  ...  2.00700  1.879   
36          IK Hakarpspojkarna 1  10.9  10.5  13.2  ...  1.99300  1.978   
87       Kouvolan Suunnistajat 1  10.9  10.5  13.2  ...  2.00100  1.986   
96               OLV Baselland 2  10.9  10.5  13.2  ...  1.98856  2.107   
97          IK Hakarpspojkarna 2  10.9  10.5  13.2  ...  1.93500  1.874   

                    log_std                                                  
leg             7         1      2      3         4      5         6      7  
team_id                                                                      
28       1.850000  0.129000  0.100  0.070  0.100000  0.110  0.091000  0.100  
36       1.838000  0.131000  0.095  0.081  0.079000  0.117  0.099000  0.083  
87       1.945000  0.100000  0.098  0.080  0.084000  0.108  0.093000  0.112  
96       1.676371  0.114143  0.100  0.100  0.149311  0.100  0.117543  0.100  
97       2.037000  0.135117  0.116  0.098  0.139937  0.124  0.086000  0.100  

[5 rows x 35 columns]

In [6]:
display(by_teams.shape)
# remove teams that have not nominated runners for all legs
by_teams = by_teams[np.all(np.isfinite(by_teams["log_mean"]), axis=1)]
display(by_teams.shape)

(407, 35)

(390, 35)

In [7]:
num_legs = shared.num_legs[ve_or_ju]
# Sum of log normal variables is not defined 
# so we simulate 10000 runs for each user and sum them and then do statistics on simulated results 
def simulate_relay_estimates(row):
    samples = pd.DataFrame()
    for i in range(1, num_legs + 1):
        if np.isnan(row["log_mean"][i]):
            print(row["log_mean"])
            print(row["name"])
        samples[i] = row["dist"][i] * lognorm.rvs(s = row["log_std"][i], scale = np.exp(row["log_mean"][i]), size = 10000)

    samples_sums = pd.DataFrame()
    # leg_1 
    # leg_1 + leg_2
    # leg_1 + leg_2 + leg_3
    # ...
    for i in range(1,num_legs + 1):
        samples_sums[i] = np.sum([ samples[j] for j in range(1,i+1) ], axis=0)

    start95 = samples_sums.quantile(0.025)
    end95 = samples_sums.quantile(0.975)
    medians = samples_sums.median()
    means = samples_sums.mean()
    
    sum_logs = np.log(samples_sums)
    sum_log_means = np.mean(sum_logs)
    sum_log_stds = np.std(sum_logs)
    
    """
    for i in range(1,8):
        bins = int(samples_sums[i].max() - samples_sums[i].min())
        name = row["name"][i]
        plt.title(f"{name} bins = {bins}")
        plt.hist(samples_sums[i], bins=bins)
        #plt.axvline(x=row["fin_real"][i], color="r")        
        plt.axvline(x=medians[i], color="g")
        plt.axvline(x=means[i], color="yellow")
        plt.show()    
    """
    
    """
    bins = int( (samples_sums.max().max() - samples_sums.min().min()) / 5) 
    plt.figure(figsize=(20, 6))
    plt.title(f"Whole team, bins = {bins}")
    plt.hist([samples_sums[1], samples_sums[2], samples_sums[3], samples_sums[4], samples_sums[5], samples_sums[6], samples_sums[7]], bins=bins)
    for i in range(1,8):
        #plt.axvline(x=row["fin_real"][i], color="r")
        plt.axvline(x=medians[i], color="g")

    plt.show()
    """
    
    fin_start95_dict = {f"fin_start95_{leg}" : start95.values[leg-1] for leg in range(1,8)}
    fin_end95_dict = {f"fin_end95_{leg}" : end95.values[leg-1] for leg in range(1,8)}
    fin_median_dict = {f"fin_median_{leg}" : medians.values[leg-1] for leg in range(1,8)}
    fin_mean_dict = {f"fin_mean_{leg}" : means.values[leg-1] for leg in range(1,8)}
    fin_sum_log_means_dict = {f"fin_sum_log_mean_{leg}" : sum_log_means.values[leg-1] for leg in range(1,8)}
    fin_sum_log_stds_dict = {f"fin_sum_log_std_{leg}" : sum_log_stds.values[leg-1] for leg in range(1,8)}
    new_cols = {**fin_start95_dict, **fin_end95_dict, **fin_median_dict, **fin_mean_dict, **fin_sum_log_means_dict, **fin_sum_log_stds_dict}

    #print(start95.values)
    #print(new_cols)
    return pd.Series(new_cols)

relay_estimates = by_teams.apply(simulate_relay_estimates, axis=1)
relay_estimates.head()

,fin_start95_1,fin_start95_2,fin_start95_3,fin_start95_4,fin_start95_5,fin_start95_6,fin_start95_7,fin_end95_1,fin_end95_2,fin_end95_3,...,fin_sum_log_mean_5,fin_sum_log_mean_6,fin_sum_log_mean_7,fin_sum_log_std_1,fin_sum_log_std_2,fin_sum_log_std_3,fin_sum_log_std_4,fin_sum_log_std_5,fin_sum_log_std_6,fin_sum_log_std_7
team_id,,,,,,,,,,,,,,,,,,,,,
28,58.794302,121.208476,194.639807,244.150202,298.966269,368.957594,447.621487,96.898273,167.057109,244.990529,...,5.793079,5.993954,6.180885,0.128458,0.082296,0.058827,0.051549,0.046715,0.041803,0.038642
36,54.280616,119.159995,200.830321,246.817562,301.911876,378.284510,457.324411,90.137178,164.043261,252.948640,...,5.798332,6.017307,6.197993,0.129816,0.080842,0.059107,0.050780,0.046310,0.042094,0.037698
87,69.434614,140.490377,233.330188,284.543285,339.831258,417.697396,503.382638,103.012459,185.054425,287.635547,...,5.912578,6.111083,6.294197,0.099489,0.069818,0.053188,0.046386,0.042751,0.038811,0.037355
96,58.469679,140.541237,245.856611,293.664527,349.729754,436.705411,502.994371,91.260528,188.755566,311.270858,...,5.954100,6.168090,6.303690,0.114506,0.075262,0.060535,0.056130,0.049919,0.046038,0.042217
97,61.168405,131.058056,219.484756,273.601181,325.077160,395.918753,490.855598,103.907827,186.827184,285.127008,...,5.892444,6.075129,6.280864,0.134682,0.089824,0.067559,0.060888,0.055140,0.047772,0.043227


In [8]:
# Flatten the troublesome multi-index to field_{leg} etc...
by_teams_flat = by_teams.copy()
by_teams_flat.columns = [f'{x[0]}_{x[1]}' for x in by_teams_flat.columns]
by_teams_flat.reset_index()

,team_id,team_1,team_2,team_3,team_4,team_5,team_6,team_7,dist_1,dist_2,...,log_mean_5,log_mean_6,log_mean_7,log_std_1,log_std_2,log_std_3,log_std_4,log_std_5,log_std_6,log_std_7
0,28,OLV Baselland 1,OLV Baselland 1,OLV Baselland 1,OLV Baselland 1,OLV Baselland 1,OLV Baselland 1,OLV Baselland 1,10.9,10.5,...,2.007000,1.879000,1.850000,0.129000,0.100000,0.070000,0.100000,0.110000,0.091000,0.100000
1,36,IK Hakarpspojkarna 1,IK Hakarpspojkarna 1,IK Hakarpspojkarna 1,IK Hakarpspojkarna 1,IK Hakarpspojkarna 1,IK Hakarpspojkarna 1,IK Hakarpspojkarna 1,10.9,10.5,...,1.993000,1.978000,1.838000,0.131000,0.095000,0.081000,0.079000,0.117000,0.099000,0.083000
2,87,Kouvolan Suunnistajat 1,Kouvolan Suunnistajat 1,Kouvolan Suunnistajat 1,Kouvolan Suunnistajat 1,Kouvolan Suunnistajat 1,Kouvolan Suunnistajat 1,Kouvolan Suunnistajat 1,10.9,10.5,...,2.001000,1.986000,1.945000,0.100000,0.098000,0.080000,0.084000,0.108000,0.093000,0.112000
3,96,OLV Baselland 2,OLV Baselland 2,OLV Baselland 2,OLV Baselland 2,OLV Baselland 2,OLV Baselland 2,OLV Baselland 2,10.9,10.5,...,1.988560,2.107000,1.676371,0.114143,0.100000,0.100000,0.149311,0.100000,0.117543,0.100000
4,97,IK Hakarpspojkarna 2,IK Hakarpspojkarna 2,IK Hakarpspojkarna 2,IK Hakarpspojkarna 2,IK Hakarpspojkarna 2,IK Hakarpspojkarna 2,IK Hakarpspojkarna 2,10.9,10.5,...,1.935000,1.874000,2.037000,0.135117,0.116000,0.098000,0.139937,0.124000,0.086000,0.100000
5,138,Pannon Supersonics 1,Pannon Supersonics 1,Pannon Supersonics 1,Pannon Supersonics 1,Pannon Supersonics 1,Pannon Supersonics 1,Pannon Supersonics 1,10.9,10.5,...,2.325000,2.274000,1.817044,0.122788,0.100000,0.100000,0.150990,0.167585,0.122889,0.100000
6,140,Espoon Akilles 1,Espoon Akilles 1,Espoon Akilles 1,Espoon Akilles 1,Espoon Akilles 1,Espoon Akilles 1,Espoon Akilles 1,10.9,10.5,...,2.273000,2.100000,2.165000,0.085000,0.114766,0.105000,0.158767,0.138721,0.108000,0.116000
7,143,Freidig SPK 1,Freidig SPK 1,Freidig SPK 1,Freidig SPK 1,Freidig SPK 1,Freidig SPK 1,Freidig SPK 1,10.9,10.5,...,2.227000,2.013000,2.359407,0.138575,0.194749,0.165226,0.235287,0.200753,0.224190,0.181328
8,154,Bussola OK 1,Bussola OK 1,Bussola OK 1,Bussola OK 1,Bussola OK 1,Bussola OK 1,Bussola OK 1,10.9,10.5,...,2.040524,2.193000,1.706450,0.129000,0.117000,0.102000,0.136178,0.110954,0.133030,0.100000
9,162,Catalonia Team 1,Catalonia Team 1,Catalonia Team 1,Catalonia Team 1,Catalonia Team 1,Catalonia Team 1,Catalonia Team 1,10.9,10.5,...,2.235776,2.107179,2.126000,0.105971,0.100000,0.113718,0.153436,0.118647,0.140743,0.100000


In [9]:
estimates = pd.concat([by_teams_flat, relay_estimates], axis=1, join='inner')

In [10]:
estimates.head()

,team_1,team_2,team_3,team_4,team_5,team_6,team_7,dist_1,dist_2,dist_3,...,fin_sum_log_mean_5,fin_sum_log_mean_6,fin_sum_log_mean_7,fin_sum_log_std_1,fin_sum_log_std_2,fin_sum_log_std_3,fin_sum_log_std_4,fin_sum_log_std_5,fin_sum_log_std_6,fin_sum_log_std_7
team_id,,,,,,,,,,,,,,,,,,,,,
28,OLV Baselland 1,OLV Baselland 1,OLV Baselland 1,OLV Baselland 1,OLV Baselland 1,OLV Baselland 1,OLV Baselland 1,10.9,10.5,13.2,...,5.793079,5.993954,6.180885,0.128458,0.082296,0.058827,0.051549,0.046715,0.041803,0.038642
36,IK Hakarpspojkarna 1,IK Hakarpspojkarna 1,IK Hakarpspojkarna 1,IK Hakarpspojkarna 1,IK Hakarpspojkarna 1,IK Hakarpspojkarna 1,IK Hakarpspojkarna 1,10.9,10.5,13.2,...,5.798332,6.017307,6.197993,0.129816,0.080842,0.059107,0.050780,0.046310,0.042094,0.037698
87,Kouvolan Suunnistajat 1,Kouvolan Suunnistajat 1,Kouvolan Suunnistajat 1,Kouvolan Suunnistajat 1,Kouvolan Suunnistajat 1,Kouvolan Suunnistajat 1,Kouvolan Suunnistajat 1,10.9,10.5,13.2,...,5.912578,6.111083,6.294197,0.099489,0.069818,0.053188,0.046386,0.042751,0.038811,0.037355
96,OLV Baselland 2,OLV Baselland 2,OLV Baselland 2,OLV Baselland 2,OLV Baselland 2,OLV Baselland 2,OLV Baselland 2,10.9,10.5,13.2,...,5.954100,6.168090,6.303690,0.114506,0.075262,0.060535,0.056130,0.049919,0.046038,0.042217
97,IK Hakarpspojkarna 2,IK Hakarpspojkarna 2,IK Hakarpspojkarna 2,IK Hakarpspojkarna 2,IK Hakarpspojkarna 2,IK Hakarpspojkarna 2,IK Hakarpspojkarna 2,10.9,10.5,13.2,...,5.892444,6.075129,6.280864,0.134682,0.089824,0.067559,0.060888,0.055140,0.047772,0.043227


In [11]:
leg_1_cols = list(filter(lambda c: "_1" in c,estimates.columns.values))
column_base_names = list(map(lambda c: c[:-2], leg_1_cols))
runner_estimates = pd.wide_to_long(estimates.reset_index(), stubnames=column_base_names, i ="team_id", j="leg", sep = "_")
runner_estimates = runner_estimates.sort_values(by=['team_id', 'leg'])
#runner_estimates = runner_estimates.drop(['team_base_name', 'estimated_log_means', 'estimated_log_stdevs'], axis=1)
runner_estimates.head(7)

team  dist               name  log_mean  log_std  \
team_id leg                                                                
28      1    OLV Baselland 1  10.9    Timon Schweizer     1.935    0.129   
        2    OLV Baselland 1  10.5      Kaspar Hägler     1.842    0.100   
        3    OLV Baselland 1  13.2     Fabian Hertner     1.746    0.070   
        4    OLV Baselland 1   7.3  Jannis Schönleber     1.949    0.100   
        5    OLV Baselland 1   7.8   Stefan Lombriser     2.007    0.110   
        6    OLV Baselland 1  11.1       Noah Zbinden     1.879    0.091   
        7    OLV Baselland 1  12.9    Christoph Meier     1.850    0.100   

             fin_start95   fin_end95  fin_median    fin_mean  \
team_id leg                                                    
28      1      58.794302   96.898273   75.688724   76.218226   
        2     121.208476  167.057109  142.364306  142.811895   
        3     194.639807  244.990529  218.160341  218.529787   
        4     244.150202  298.190935  269.851516  270.150970   
        5     298.966269  359.436698  328.024369  328.379428   
        6     368.957594  435.123815  401.033669  401.347524   
        7     447.621487  520.693027  483.346288  483.780420   

             fin_sum_log_mean  fin_sum_log_std  
team_id leg                                     
28      1            4.325367         0.128458  
        2            4.958143         0.082296  
        3            5.385192         0.058827  
        4            5.597653         0.051549  
        5            5.793079         0.046715  
        6            5.993954         0.041803  
        7            6.180885         0.038642

In [12]:
runner_estimates[runner_estimates['team'].str.contains("Reak")][["name", "fin_start95", "fin_median", "fin_end95"]]

name  fin_start95  fin_median   fin_end95
team_id leg                                                         
530     1        Mikko Peltonen    82.423893   99.603783  120.842671
        2         Jeppe Koivula   158.884525  180.580999  205.702170
        3          Joni Freeman   265.373514  297.030448  333.442712
        4       Pasi Huhtiniemi   328.291893  362.763433  401.088335
        5          Tommi Pajala   392.662707  431.964740  473.215999
        6         Olavi Kanerva   493.375126  542.212373  596.362897
        7       Janne Vaittinen   612.837383  678.339891  754.838521
1201    1         Jyri Kytömäki    83.688218  107.179501  136.997810
        2             Mikko Apo   183.680684  219.229469  262.421762
        3           Aki Kaivola   316.357404  360.309451  412.621401
        4            Mika Tuoma   385.683328  436.112238  493.862619
        5    Oskari Pirttikoski   483.007724  540.339892  606.295778
        6            Pasi Koski   605.459591  669.721142  744.368162
        7    Antti-Ville Jokela   727.220853  798.061504  880.040069

In [13]:
# Convert minutes to date and times
start_timestamp = shared.start_timestamp[ve_or_ju][2019]

runner_estimates["fin_time_median"] = pd.to_datetime(runner_estimates["fin_median"] * 60, unit = "s", origin= start_timestamp)
runner_estimates["fin_time_start95"] = pd.to_datetime(runner_estimates["fin_start95"] * 60, unit = "s", origin= start_timestamp)
runner_estimates["fin_time_end95"] = pd.to_datetime(runner_estimates["fin_end95"] * 60, unit = "s", origin= start_timestamp)

runner_estimates.head()

team  dist               name  log_mean  log_std  \
team_id leg                                                                
28      1    OLV Baselland 1  10.9    Timon Schweizer     1.935    0.129   
        2    OLV Baselland 1  10.5      Kaspar Hägler     1.842    0.100   
        3    OLV Baselland 1  13.2     Fabian Hertner     1.746    0.070   
        4    OLV Baselland 1   7.3  Jannis Schönleber     1.949    0.100   
        5    OLV Baselland 1   7.8   Stefan Lombriser     2.007    0.110   

             fin_start95   fin_end95  fin_median    fin_mean  \
team_id leg                                                    
28      1      58.794302   96.898273   75.688724   76.218226   
        2     121.208476  167.057109  142.364306  142.811895   
        3     194.639807  244.990529  218.160341  218.529787   
        4     244.150202  298.190935  269.851516  270.150970   
        5     298.966269  359.436698  328.024369  328.379428   

             fin_sum_log_mean  fin_sum_log_std               fin_time_median  \
team_id leg                                                                    
28      1            4.325367         0.128458 2019-06-16 00:15:41.323441029   
        2            4.958143         0.082296 2019-06-16 01:22:21.858368158   
        3            5.385192         0.058827 2019-06-16 02:38:09.620459318   
        4            5.597653         0.051549 2019-06-16 03:29:51.090946674   
        5            5.793079         0.046715 2019-06-16 04:28:01.462153673   

                         fin_time_start95                fin_time_end95  
team_id leg                                                              
28      1   2019-06-15 23:58:47.658141851 2019-06-16 00:36:53.896354198  
        2   2019-06-16 01:01:12.508570194 2019-06-16 01:47:03.426550150  
        3   2019-06-16 02:14:38.388442993 2019-06-16 03:04:59.431740046  
        4   2019-06-16 03:04:09.012129307 2019-06-16 03:58:11.456071854  
        5   2019-06-16 03:58:57.976163149 2019-06-16 04:59:26.201881170

In [14]:

for_print = runner_estimates.copy()
for_print = for_print.reset_index()
for_print = for_print.set_index('team_id')
for_print = for_print.round(2)
for_print["fin_time_median"] = for_print["fin_time_median"].dt.strftime("%H:%M")
for_print["fin_time_start95"] = for_print["fin_time_start95"].dt.strftime("%H:%M")
for_print["fin_time_end95"] = for_print["fin_time_end95"].dt.strftime("%H:%M")
for_print = for_print[['team',
 'leg',
 'name', 
 'fin_time_start95', 
 'fin_time_end95',
 'fin_time_median']]

for_print[for_print['team'].str.contains("Reak")]

,team,leg,name,fin_time_start95,fin_time_end95,fin_time_median
team_id,,,,,,
530,Reaktor Innovations 1,1,Mikko Peltonen,00:22,01:00,00:39
530,Reaktor Innovations 1,2,Jeppe Koivula,01:38,02:25,02:00
530,Reaktor Innovations 1,3,Joni Freeman,03:25,04:33,03:57
530,Reaktor Innovations 1,4,Pasi Huhtiniemi,04:28,05:41,05:02
530,Reaktor Innovations 1,5,Tommi Pajala,05:32,06:53,06:11
530,Reaktor Innovations 1,6,Olavi Kanerva,07:13,08:56,08:02
530,Reaktor Innovations 1,7,Janne Vaittinen,09:12,11:34,10:18
1201,Reaktor Innovations 2,1,Jyri Kytömäki,00:23,01:16,00:47
1201,Reaktor Innovations 2,2,Mikko Apo,02:03,03:22,02:39


In [28]:
for_web = runner_estimates.copy().reset_index()
for_web["fin_time_median"] = for_web["fin_time_median"].dt.tz_localize('EET').dt.tz_convert('UTC')
for_web["fin_time_start95"] = for_web["fin_time_start95"].dt.tz_localize('EET').dt.tz_convert('UTC')
for_web["fin_time_end95"] = for_web["fin_time_end95"].dt.tz_localize('EET').dt.tz_convert('UTC')

#for_web["fin_time_median"] = for_web["fin_time_median"].dt.strftime("%H:%M")
#for_web["fin_time_start95"] = for_web["fin_time_start95"].dt.strftime("%H:%M")
#for_web["fin_time_end95"] = for_web["fin_time_end95"].dt.strftime("%H:%M")
for_web = for_web[[
 'team_id',
 'leg',
 'team',
 'name',
 'fin_sum_log_mean', 
 'fin_sum_log_std',
  'fin_time_start95', 
 'fin_time_end95',
 'fin_time_median']]
for_web[for_web['team'].str.contains("Reak")]

,team_id,leg,team,name,fin_sum_log_mean,fin_sum_log_std,fin_time_start95,fin_time_end95,fin_time_median
378,530,1,Reaktor Innovations 1,Mikko Peltonen,4.601768,0.097625,2019-06-15 21:22:25.433609009+00:00,2019-06-15 22:00:50.560231447+00:00,2019-06-15 21:39:36.226990461+00:00
379,530,2,Reaktor Innovations 1,Jeppe Koivula,5.196859,0.065718,2019-06-15 22:38:53.071474552+00:00,2019-06-15 23:25:42.130189656+00:00,2019-06-15 23:00:34.859918118+00:00
380,530,3,Reaktor Innovations 1,Joni Freeman,5.694214,0.057828,2019-06-16 00:25:22.410827160+00:00,2019-06-16 01:33:26.562724352+00:00,2019-06-16 00:57:01.826868773+00:00
381,530,4,Reaktor Innovations 1,Pasi Huhtiniemi,5.893904,0.050760,2019-06-16 01:28:17.513559817+00:00,2019-06-16 02:41:05.300117731+00:00,2019-06-16 02:02:45.805959940+00:00
382,530,5,Reaktor Innovations 1,Tommi Pajala,6.068146,0.047481,2019-06-16 02:32:39.762405872+00:00,2019-06-16 03:53:12.959912062+00:00,2019-06-16 03:11:57.884389639+00:00
383,530,6,Reaktor Innovations 1,Olavi Kanerva,6.295530,0.048764,2019-06-16 04:13:22.507532835+00:00,2019-06-16 05:56:21.773802042+00:00,2019-06-16 05:02:12.742402554+00:00
384,530,7,Reaktor Innovations 1,Janne Vaittinen,6.519623,0.052529,2019-06-16 06:12:50.242981672+00:00,2019-06-16 08:34:50.311274052+00:00,2019-06-16 07:18:20.393459558+00:00
1358,1201,1,Reaktor Innovations 2,Jyri Kytömäki,4.673784,0.125265,2019-06-15 21:23:41.293090820+00:00,2019-06-15 22:16:59.868601799+00:00,2019-06-15 21:47:10.770049572+00:00
1359,1201,2,Reaktor Innovations 2,Mikko Apo,5.390681,0.090700,2019-06-15 23:03:40.841041327+00:00,2019-06-16 00:22:25.305738926+00:00,2019-06-15 23:39:13.768154383+00:00
1360,1201,3,Reaktor Innovations 2,Aki Kaivola,5.888210,0.067644,2019-06-16 01:16:21.444264650+00:00,2019-06-16 02:52:37.284070730+00:00,2019-06-16 02:00:18.567082644+00:00


In [29]:
for_web.to_json(f'web-lib/for_web_{ve_or_ju}_2019.json', orient="records", date_format="iso")